In [1]:
import pandas as pd 
import numpy as np

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies=movies.merge(credits,on='title')

In [4]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
movies.isnull().sum()
movies.dropna(inplace=True)
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [6]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [7]:
movies['genres']=movies['genres'].apply(convert)

In [8]:
movies['keywords']=movies['keywords'].apply(convert)

In [9]:

def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [10]:
movies['cast']=movies['cast'].apply(convert3)

In [11]:

def director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [12]:
movies['crew']=movies['crew'].apply(director)

In [13]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [14]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])


In [15]:
movies['tags']=movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew'] + movies['overview']

In [16]:
new_df=movies[['movie_id','title','tags']]

In [17]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\Sushma\AppData\Local\Temp\ipykernel_11728\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [18]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\Sushma\AppData\Local\Temp\ipykernel_11728\4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [20]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [21]:
print(cv.get_feature_names_out())

['000' '007' '10' ... 'zone' 'zoo' 'zooeydeschanel']


In [27]:
import nltk
from nltk.stem.porter import PorterStemmer 
ps=PorterStemmer()
def stem(text):
    if isinstance(text, str):
        y = []
        for i in text.split():
            y.append(ps.stem(i))
        return " ".join(y)
    else:
        return ""

    

In [28]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\Sushma\AppData\Local\Temp\ipykernel_11728\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [30]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      shape=(5000,), dtype=object)

In [ ]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4806, 5000))

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
similiarity=cosine_similarity(vectors)

In [35]:
similiarity[3]

array([0.03823596, 0.02116669, 0.04233338, ..., 0.05270463, 0.0559017 ,
       0.05735393], shape=(4806,))

In [45]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similiarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)


In [47]:
recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman
Batman & Robin


In [48]:
new_df

,movie_id,title,tags
0,19995,Avatar,action adventur fantasi sciencefict culturecla...
1,285,Pirates of the Caribbean: At World's End,adventur fantasi action ocean drugabus exotici...
2,206647,Spectre,action adventur crime spi basedonnovel secreta...
3,49026,The Dark Knight Rises,action crime drama thriller dccomic crimefight...
4,49529,John Carter,action adventur sciencefict basedonnovel mar m...
...,...,...,...
4804,9367,El Mariachi,action crime thriller unitedstates–mexicobarri...
4805,72766,Newlyweds,comedi romanc edwardburn kerrybishé marshadiet...
4806,231617,"Signed, Sealed, Delivered",comedi drama romanc tvmovi date loveatfirstsig...
4807,126186,Shanghai Calling,danielhenney elizacoup billpaxton danielhsia w...
